In [ ]:
'''
    Made By: Christopher Fusco
    Made On: Boxing day, 2021
    Made With: A lot of whiskey
'''

In [ ]:
# Installs
%pip install pandas
%pip install bs4
%pip install requests
%pip install tqdm

In [ ]:
# Imports
import pandas as pd
import requests
import unicodedata
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
# Variables
OUTPUT_FILE = 'enzymes.csv'

In [ ]:
# Gets all enzymes and their information at https://www.nebiolabs.com.au/products/restriction-endonucleases/hf-nicking-master-mix-time-saver-other.
# Data is saved to file.
def get_and_print_data():
    df = pd.DataFrame()

    req = requests.get('https://www.nebiolabs.com.au/products/restriction-endonucleases/hf-nicking-master-mix-time-saver-other')
    soup = BeautifulSoup(req.text, 'html.parser')
    sections = soup.find_all('section', attrs={'class': 'category'})
    for i in range(1, 6):
        sections_children = sections[i].find_all('section')

        print('Obtaining section: ' + sections[i].h2.text)
        for section in tqdm(sections_children):
            try:
                child_req = requests.get('https://www.nebiolabs.com.au' + section.h3.a['href'])
                child_soup = BeautifulSoup(child_req.text, 'html.parser')

                name = section.h3.a['name'].replace('®', '')
                cut_site = 'None'

                for li in child_soup.find_all('ul')[12].find_all('li'):
                    pretty = unicodedata.normalize("NFKD", li.text)
                    if 'Restriction Enzyme Cut Site: ' in pretty:
                        cut_site = pretty.replace('Restriction Enzyme Cut Site: ', '')

                table = child_soup.find('table', attrs={'class': 'js-pdp-product-grid'})
                units = unicodedata.normalize("NFKD", table.tbody.tr.find_all('td')[2].span.text).replace(' ', '').replace('units', '')
                price = table.tbody.tr.find_all('td')[3].span['data-price']

                df = df.append({'section': sections[i].h2.text, 'name': name, 'cut site': cut_site , 'units': units, 'price ($)': price}, ignore_index=True)
            except:
                name = section.h3.a['name']
                print(f'There was an error processing {name}')
    df.to_csv(OUTPUT_FILE, index=False)
    print('Completed.')

In [ ]:
# Run the code
get_and_print_data()